In [4]:
import pandas as pd
import sqlite3

# u.data : user_id, movie_id, rating, timestamp (tab-separated)
ratings_df = pd.read_csv("../data/u.data", sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# u.item : movie_id | title | release_date | video_release_date | IMDb URL | genres (19 colonnes 0/1)
columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)]
movies_df = pd.read_csv("../data/u.item", sep='|', names=columns, encoding='latin-1')

# Connexion SQLite
conn = sqlite3.connect("movielens_100k.db")

# Insérer dans SQLite
movies_df.to_sql("movies", conn, if_exists="replace", index=False)
ratings_df.to_sql("ratings", conn, if_exists="replace", index=False)

100000

In [ ]:
# Movies rate by a user
user_id = 1
query_user_movies = f"""
SELECT m.movie_id, m.title, r.rating
FROM ratings r
JOIN movies m ON r.movie_id = m.movie_id
WHERE r.user_id = {user_id}
"""
user_movies = pd.read_sql_query(query_user_movies, conn)
print(user_movies.head())

   movie_id                       title  rating
0        61  Three Colors: White (1994)       4
1       189     Grand Day Out, A (1992)       3
2        33            Desperado (1995)       4
3       160  Glengarry Glen Ross (1992)       4
4        20   Angels and Insects (1995)       4


In [ ]:
# average rating per movie
query_avg = """
SELECT movie_id, AVG(rating) AS avg_rating
FROM ratings
GROUP BY movie_id
HAVING COUNT(rating) >= 5
ORDER BY avg_rating DESC
"""
top_movies = pd.read_sql_query(query_avg, conn)
print(top_movies.head())

   movie_id  avg_rating
0      1449    4.625000
1       408    4.491071
2       318    4.466443
3       169    4.466102
4       483    4.456790


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Choisir un film préféré
fav_movie = movies_df[movies_df['title'] == 'Toy Story (1995)']
fav_vector = fav_movie.iloc[:, 5:].values  # colonnes genre_0 à genre_18

# Similarité avec tous les films
similarities = cosine_similarity(fav_vector, movies_df.iloc[:, 5:].values)
similar_indices = similarities[0].argsort()[::-1][1:6]  # top 5
recommended_titles = movies_df.iloc[similar_indices]['title'].tolist()
print("Films similaires à 'Toy Story (1995)':")
print(recommended_titles)

Films similaires à 'Toy Story (1995)':
['Aladdin and the King of Thieves (1996)', 'Aladdin (1992)', 'Goofy Movie, A (1995)', 'Love Bug, The (1969)', 'Santa Clause, The (1994)']


In [9]:
import pandas as pd
import sqlite3

# --- Chemins vers les fichiers MovieLens 100k ---
RATINGS_FILE = "../data/u.data"
MOVIES_FILE = "../data/u.item"
DB_FILE = "../data/movielens_100k.db"

# --- Charger les ratings ---
# u.data : user_id \t movie_id \t rating \t timestamp
ratings_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(RATINGS_FILE, sep="\t", names=ratings_cols, encoding="latin-1")
ratings = ratings.drop(columns=["timestamp"])  # on n'a pas besoin du timestamp

# --- Charger les films ---
# u.item : movie_id | title | release_date | video_release_date | IMDb URL | genres (19 colonnes)
movies_cols = [
    "movie_id", "title", "release_date", "video_release_date", "IMDb_URL",
    "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
    "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies = pd.read_csv(MOVIES_FILE, sep="|", names=movies_cols, encoding="latin-1")

# --- Ajouter une colonne genres concaténées pour NLP / TF-IDF ---
genre_cols = movies_cols[5:]  # les 19 colonnes de genres
movies["genres"] = movies[genre_cols].apply(lambda x: " ".join([genre for genre, flag in zip(genre_cols, x) if flag == 1]), axis=1)

# --- Connexion SQLite ---
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# --- Créer tables ---
cursor.execute("DROP TABLE IF EXISTS movies")
cursor.execute("DROP TABLE IF EXISTS ratings")

# --- Table movies ---
movies_to_save = movies[["movie_id", "title", "genres"]]
movies_to_save.to_sql("movies", conn, index=False)

# --- Table ratings ---
ratings.to_sql("ratings", conn, index=False)

conn.commit()
conn.close()

print(f"Base SQLite créée avec succès : {DB_FILE}")

Base SQLite créée avec succès : ../data/movielens_100k.db


In [ ]:
import sqlite3
conn = sqlite3.connect(r"C:\Users\bapti\MovieRecommenderSystem\data\movielens_100k.db")
print("OK")

OK
